In [1]:
import pandas as pd
import json
import urllib.request
from IPython.display import display, HTML

In [2]:
# Enter url from API to extract JSON response from
base_url = r'https://aqs.epa.gov/data/api/sampleData/bySite?email=fbarak@albany.edu&key=silverfox11&param=43202,43203,43206,43204,43205,43207,43208,43212,43214,43216,43217,43218,43220,43221,43224,43226,43227,43230,43231,43232,43233,43235,43238,43242,43243,43244,43247,43248,43249,43250,43252,43253,43261,43262,43263,43280,43284,43285,43291,43372,43502,43503,43552,43801,43802,43803,43804,43811,43812,43813,43814,43815,43817,43818,43819,43820,43823,43824,43826,43828,43829,43830,43831,43843,43844,43860,43954,43960,45109,45201,45202,45203,45204,45207,45208,45209,45210,45211,45212,45213,45218,45219,45220,45225,45801,45805,45806,45807,45809,45810,&bdate=20200101&edate=20201231&state=36&county=081&site=0124'

df = pd.DataFrame(columns=['date_local', 'time_local', 'detection_limit', 'parameter', 'sample_measurement', 'units_of_measure'])

# Define which years to extract here
for year in range(2020, 2022):
    # Construct the URL for the current year
    bdate = f'{year}0101'
    edate = f'{year}1231'
    url = f'{base_url}&bdate={bdate}&edate={edate}'
    
    try:
        # Retrieve data from the API and add to the DataFrame
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read().decode())       
        
        #Error checker for URL opening
        if (data['Header'][0]['status'] != "Success"):
            raise Exception("Error. Failed to retrieve data for year ",bdate,".")
        else:
            print("URL for data range ", bdate, " successfully opened")
        
        for item in data['Data']:
            df = df.append({
                'date_local': item['date_local'],
                'time_local': item['time_local'],
                'detection_limit': item['detection_limit'],
                'parameter': item['parameter'],
                'sample_measurement': item['sample_measurement'],
                'units_of_measure': item['units_of_measure']
            }, ignore_index=True)
    except:
        print("Error. An exception occured in the selected period. Try date range between ",bdate," and ", edate, " again.")

display(df)

URL for data range  20200101  successfully opened
URL for data range  20210101  successfully opened


,date_local,time_local,detection_limit,parameter,sample_measurement,units_of_measure
0,2020-01-04,00:00,0.1000,Methylcyclopentane,0.9,Parts per billion Carbon
1,2020-01-10,00:00,0.1000,Methylcyclopentane,0.4,Parts per billion Carbon
2,2020-01-16,00:00,0.1000,Methylcyclopentane,0.3,Parts per billion Carbon
3,2020-01-22,00:00,0.1000,Methylcyclopentane,0.7,Parts per billion Carbon
4,2020-01-28,00:00,0.1000,Methylcyclopentane,0.2,Parts per billion Carbon
...,...,...,...,...,...,...
8474,2021-09-01,00:00,0.0474,"1,2,4-Trichlorobenzene",0.0,Parts per billion Carbon
8475,2021-09-07,00:00,0.0474,"1,2,4-Trichlorobenzene",0.0,Parts per billion Carbon
8476,2021-09-13,00:00,0.0474,"1,2,4-Trichlorobenzene",0.0,Parts per billion Carbon
8477,2021-09-19,00:00,0.0474,"1,2,4-Trichlorobenzene",0.0,Parts per billion Carbon


In [3]:
# Convert to correct format

df['datetime'] = pd.to_datetime(df['date_local']+' '+df['time_local'], format='%Y-%m-%d %H:%M')

pivot_df = pd.pivot_table(df, values='sample_measurement', index=['date_local'], columns=['parameter'], aggfunc='sum')
pivot_df_detectLim = pd.pivot_table(df, values='detection_limit', index=['date_local'], columns=['parameter'], aggfunc='sum')
pivot_df_detectLim.columns = [col + '_dl' for col in pivot_df.columns]


# Concatenate both pivot DF's:
pivoted_final = pd.concat([pivot_df, pivot_df_detectLim], axis=1)

display(pivoted_final)

,"1,1,2,2-Tetrachloroethane","1,1,2-Trichloroethane","1,1-Dichloroethane","1,1-Dichloroethylene","1,2,3-Trimethylbenzene","1,2,4-Trichlorobenzene","1,2,4-Trimethylbenzene","1,2-Dichlorobenzene","1,2-Dichloropropane","1,3,5-Trimethylbenzene",...,n-Pentane_dl,n-Propylbenzene_dl,n-Undecane_dl,o-Ethyltoluene_dl,o-Xylene_dl,p-Diethylbenzene_dl,p-Ethyltoluene_dl,"trans-1,3-Dichloropropene_dl",trans-2-Butene_dl,trans-2-Pentene_dl
date_local,,,,,,,,,,,,,,,,,,,,,
2020-01-04,0.0,0.0,0.0,0.0,0.1,0.0,1.4,0.0,0.0,0.3,...,0.1,0.1,0.1,0.1,0.2480,0.1,0.1,0.0273,0.1,0.1
2020-01-10,0.0,0.0,0.0,0.0,0.1,0.0,0.9,0.0,0.0,0.3,...,0.1,0.1,0.1,0.1,0.2480,0.1,0.1,0.0273,0.1,0.1
2020-01-16,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.1,...,0.1,0.1,0.1,0.1,0.2480,0.1,0.1,0.0273,0.1,0.1
2020-01-22,0.0,0.0,0.0,0.0,0.1,0.0,1.4,0.0,0.0,0.3,...,0.1,0.1,0.1,0.1,0.2480,0.1,0.1,0.0273,0.1,0.1
2020-01-28,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.2480,0.1,0.1,0.0273,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-06,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.2,...,NaN,NaN,NaN,NaN,0.0632,NaN,NaN,0.0276,NaN,NaN
2021-12-12,0.0,0.0,0.0,0.0,NaN,0.0,0.5,0.0,0.0,0.1,...,NaN,NaN,NaN,NaN,0.0632,NaN,NaN,0.0276,NaN,NaN
2021-12-18,0.0,0.0,0.0,0.0,NaN,0.0,0.6,0.0,0.0,0.2,...,NaN,NaN,NaN,NaN,0.0632,NaN,NaN,0.0276,NaN,NaN


In [4]:
# Save to CSV
pivoted_final.to_csv(r"C:\Users\fbara\OneDrive - University at Albany - SUNY\EPA\sampleAPIFile_pivoted.csv")